In [16]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys
from tqdm import tqdm

In [2]:
zz=openai.Model.list()

In [3]:
len(zz['data'])

62

In [4]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

In [5]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load prompt_template.md

In [6]:
sys.path.append('..')
from utils import *
from functools import partial

In [7]:
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [8]:
prompt_dict().keys()

dict_keys(['Preamble', 'Problem-solver', 'Conversation summarizer', 'Construct Kinetic Hamiltonian (continuum version, single-particle)', 'Construct Kinetic Hamiltonian (continuum version, second-quantized)', 'Construct Kinetic Hamiltonian (lattice version)', 'Define each term in Kinetic Hamiltonian (continuum version)', 'Construct Potential Hamiltonian (continuum version)', 'Define each term in Potential Hamiltonian (continuum version)', 'Construct interaction Hamiltonian (real space, lattice version)', 'Construct interaction Hamiltonian (momentum space)', 'Convert from single-particle to second-quantized form, return in matrix', 'Convert from single-particle to second-quantized form, return in summation (expand the matrix)', 'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)', 'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)', 'Convert interacting Hamiltonian in real space to momentum space (lattice version)', 

# Prompt fill place holder

In [9]:
import json

In [10]:
with open('1106.6060.jsonl','r') as f:
    kwargs_jsonl= [json.loads(line) for line in f]

In [11]:
def load_excerpt(sources):
    excerpt=''
    for tex, lines in sources.items():
        with open(tex,'r') as f:
            f_list=list(f)
            for line in lines:
                excerpt=excerpt+''.join(f_list[line[0]:line[1]])
    return excerpt

In [12]:
prompt_system='''I will provide you a Excerpt of physics paper, and a Template. You will need to fill the placeholders in the template using the correct information from the excerpt.
Here are conventions: 
{..} means a placeholder which you need to fill by extracting information from the excerpt.
{A|B} means you need to make a choice between A and B
[..] means optional sentence. You should decide whether to use it depending on the excerpt.
{{..}} DOES NOT mean a placeholder. You should not change the content inside double curly braces {{..}}.
`You should recall that {expression_second_nonint}.` : this sentence should be kept as is.


Finally, if you cannot figure out the placeholder, you should leave it as is.'''

In [13]:
def drop_text_after(string):
    string_new=''
    for line in string.split('\n'):
        if '===' in line:
            break
        else:
            if 'You should recall that' not in line:
                string_new='\n'.join([string_new,line])
    return string_new[1:]


In [14]:
def extractor(descriptor,message=False):
    sys_msg=[{'role': 'system', 'content': prompt_system}]
    question_prompt='\nTemplate:\n {template} \n\n Excerpt:\n {excerpt}'.format(template=drop_text_after(prompt_dict()[descriptor['task']]), excerpt=load_excerpt(descriptor['source']))
    user_msg=[{'role':'user','content':question_prompt}]
    messages = sys_msg + user_msg
    if message:
        return sys_msg[0]['content']+user_msg[0]['content']
    rs= openai.ChatCompletion.create(messages=messages, **model_params)
    response=rs['choices'][0]['message'].content
    return response


In [17]:
string=''
for kwargs in tqdm(kwargs_jsonl):
    string+='# '+kwargs['task']+'\n'
    response=(extractor(kwargs))
    string+=response+'\n\n'
    
    

100%|██████████| 33/33 [14:01<00:00, 25.51s/it]


In [18]:
print(string)

# Construct Kinetic Hamiltonian (lattice version)
You will be instructed to describe the kinetic term of Hamiltonian in the CuO$_2$ plane in the real space in the second-quantized form.   
The degrees of freedom of the system are: spin $s$ and site $i$     
The kinetic term is a tight-binding model composed of the following hopping process: 
between copper $d_{x^2-y^2}$ orbital at site $i$ and oxygen $p_\nu$ orbital at site $i+\hat{\nu}/2$ with the amplitude $t_{pd}$, and between neighboring oxygen sites with the amplitude $t_{pp}$.
You should ensure the hermiticity of the Hamiltonian.
The summation should be taken over all spin $s$ and all real space positions.  
Return the Kinetic Hamiltonian $\HH_{0}$.

Use the following conventions for the symbols:  
$\hat{d}^{\dag}_{i,s}$ creates a hole in the copper $d_{x^2-y^2}$ orbital at site $i$ with spin $s$,
$\hat{p}^{\dag}_{ i+\hat{\nu}/2, s}$ creates a hole in the oxygen $p_\nu$ orbital at the site $i+\hat{\nu}/2$ for $\nu = x, y$, and
$\

In [19]:
pwd=os.getcwd()

In [20]:
with open(pwd.split('/')[-1]+'_extractor.md','w') as f:
    f.write(string)